# **Distribución de créditos por segmento**

In [2]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo',
 'df_financiamientos_2024_tulancingo']

In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## 2022

In [4]:
df_financiamientos_2022_tulancingo= df_financiamientos_2022_tulancingo[['id','monto']]

In [5]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_tulancingo['segmento'] = df_financiamientos_2022_tulancingo['monto'].apply(asignar_segmento)
df_financiamientos_2022_tulancingo.head()


,id,monto,segmento
0,1,0.00,E
1,2,0.00,E
2,3,14398.00,E
3,4,96283.00,E
4,5,159720.57,E


In [6]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = df_financiamientos_2022_tulancingo['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(df_financiamientos_2022_tulancingo)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2022 = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2022.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2022

,Segmento,Cantidad,Porcentaje
4,A,2,0.632911
3,B,8,2.531646
2,C,19,6.012658
1,D,68,21.518987
0,E,217,68.670886
5,S,2,0.632911


In [7]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Datos
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#E50CB6']
porcentajes = creditos_segmentos_2022['Porcentaje']
labels = creditos_segmentos_2022['Segmento']

# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(
    textinfo='percent+label',  # Mostrar tanto porcentaje como etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.9,  # Posición horizontal
        y=0.5,  # Posición vertical
        font=dict(size=18)
    ),
    showlegend=True              # Mostrar la leyenda (puedes cambiar a False si no deseas mostrarla)
)

# Función para exportar la gráfica como archivo HTML con configuración personalizada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón de guardar como imagen
            'resetScale2d',  # Reset de ejes (no aplica aquí, pero lo incluimos como ejemplo)
            'zoom2d',        # Botón de zoom
            'pan2d',         # Botón de desplazamiento
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2022', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()


In [8]:
# Datos proporcionados
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#E50CB6', '#A9A9A9']  # Añadimos gris
porcentajes = creditos_segmentos_2022['Porcentaje']
labels = creditos_segmentos_2022['Segmento']

# Asignar un color gris para los segmentos que no tengan porcentaje
# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in labels.values else '#A9A9A9' 
                     for i, segmento in enumerate(segmentos)}

# Ordenar los segmentos según el orden deseado
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE"]

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes[labels == segmento].sum() if segmento in labels.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent+label',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Función para exportar la gráfica como archivo HTML con configuración personalizada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2022_OFICIAL', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2023

In [9]:
df_financiamientos_2023_tulancingo= df_financiamientos_2023_tulancingo[['id','monto']]

In [10]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_tulancingo['segmento'] = df_financiamientos_2023_tulancingo['monto'].apply(asignar_segmento)
df_financiamientos_2023_tulancingo.head()


,id,monto,segmento
0,1,1585485.28,D
1,2,608908.05,E
2,3,1918694.86,C
3,4,0.00,E
4,5,0.00,E


In [11]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = df_financiamientos_2023_tulancingo['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(df_financiamientos_2023_tulancingo)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2023 = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2023.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2023

,Segmento,Cantidad,Porcentaje
5,A,2,0.641026
3,B,4,1.282051
2,C,23,7.371795
1,D,77,24.679487
0,E,203,65.064103
4,S,3,0.961538


In [12]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

porcentajes = creditos_segmentos_2023['Porcentaje']
labels = creditos_segmentos_2023['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2023',  carpeta='assets/graficas')

fig.show()

In [13]:
# Datos proporcionados
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#E50CB6', '#A9A9A9']  # Añadimos gris

porcentajes = creditos_segmentos_2023['Porcentaje']
labels = creditos_segmentos_2023['Segmento']

# Asignar un color gris para los segmentos que no tengan porcentaje
# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in labels.values else '#A9A9A9' 
                     for i, segmento in enumerate(segmentos)}

# Ordenar los segmentos según el orden deseado
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE"]

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes[labels == segmento].sum() if segmento in labels.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent+label',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
    xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Función para exportar la gráfica como archivo HTML con configuración personalizada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón de guardar como imagen
            'resetScale2d',  # Reset de ejes (no aplica aquí, pero lo incluimos como ejemplo)
            'zoom2d',        # Botón de zoom
            'pan2d',         # Botón de desplazamiento
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2023_OFICIAL', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2024

In [14]:
df_financiamientos_2024_tulancingo= df_financiamientos_2024_tulancingo[['id','monto']]

In [15]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_tulancingo['segmento'] = df_financiamientos_2024_tulancingo['monto'].apply(asignar_segmento)
df_financiamientos_2024_tulancingo.head()


,id,monto,segmento
0,1,479825.86,E
1,2,527818.15,E
2,3,47387.00,E
3,4,0.00,E
4,5,0.00,E


In [16]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = df_financiamientos_2024_tulancingo['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(df_financiamientos_2024_tulancingo)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2023 = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2023.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2023

,Segmento,Cantidad,Porcentaje
4,A,2,1.015228
3,B,7,3.553299
2,C,9,4.568528
1,D,32,16.243655
0,E,145,73.604061
5,S,2,1.015228


In [17]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

porcentajes = creditos_segmentos_2023['Porcentaje']
labels = creditos_segmentos_2023['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2024',  carpeta='assets/graficas')

fig.show()

In [18]:
# Datos proporcionados
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#E50CB6', '#A9A9A9']  # Añadimos gris

porcentajes = creditos_segmentos_2023['Porcentaje']
labels = creditos_segmentos_2023['Segmento']

# Asignar un color gris para los segmentos que no tengan porcentaje
# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in labels.values else '#A9A9A9' 
                     for i, segmento in enumerate(segmentos)}

# Ordenar los segmentos según el orden deseado
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE"]

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes[labels == segmento].sum() if segmento in labels.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent+label',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ),
    xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    autosize=True,
    showlegend=True  # Mostrar la leyenda
)

# Función para exportar la gráfica como archivo HTML con configuración personalizada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón de guardar como imagen
            'resetScale2d',  # Reset de ejes (no aplica aquí, pero lo incluimos como ejemplo)
            'zoom2d',        # Botón de zoom
            'pan2d',         # Botón de desplazamiento
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2024_OFICIAL', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()